<a href="https://colab.research.google.com/github/phenix-project/Colabs/blob/main/alphafold2/AlphaFoldPhenixGUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#@markdown <b><font color='green'><h1 style="font-family:verdana;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  AlphaFold from the Phenix GUI</font></b>

#@markdown <p><center><b><font color='blue'> This Colab is no longer in supported (Colab changes versions of software too frequently for us to keep up). </p><p>Please use the Phenix server from the Phenix GUI instead.
